In [1]:
#Data source
dpath = "G:/Lire/"
import pandas as pd, numpy as np

#Preprocessing steps
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.neural_network import MLPClassifier

#Metrics to use
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
#List of all classifiers that will be trained
classifiers = [LogisticRegression(), RandomForestClassifier(), GaussianNB()]

In [3]:
#List of all features that can be combined in early fusion
feat_to_combine = ["FCTH", "JCD", "Tamura"]

In [4]:
#All possible combinations of the features in the list
from itertools import combinations
feature_subsets = []
for L in range(1, len(feat_to_combine)+1):
    for subset in combinations(feat_to_combine, L):
        feature_subsets.append(subset)
        
print(feature_subsets)

[('FCTH',), ('JCD',), ('Tamura',), ('FCTH', 'JCD'), ('FCTH', 'Tamura'), ('JCD', 'Tamura'), ('FCTH', 'JCD', 'Tamura')]


In [5]:
#Function to combine features using dataframe merge

def combining_features(filelist, feature_set, set_type="train"):
    features = pd.read_csv(filelist)

    for feat in feature_set:
        name = "features_%s_%s.csv" % (feat, set_type)
        df = pd.read_csv(dpath + name)
        features["target"] = df["target"]
        features = features.merge(df, how="inner", on=["filename", "target"])

    print("Set %s contains %s features" % (feature_set, features.shape[1]))
    return features

In [6]:
#Function to obtain the classifier name from their string representation
def classifier_name(classifier):
    return str(classifier).split("(")[0]

In [7]:
%%time

metrics = pd.DataFrame(columns=["subset", "model", "accuracy", "F1" ,"precision", "recall"])

for feat_set in feature_subsets:
    print("--- Processing Subset:", feat_set)
    
    #Combining Features in the train and test set
    features_train = combining_features(dpath + "filenames_train.csv", feat_set, set_type="train")
    features_test = combining_features(dpath + "filenames_test.csv", feat_set, set_type="test")
    
    #Separating ids and target from the datasets
    id_train = features_train["filename"]
    y_train = features_train["target"]
    features_train = features_train.drop(["filename", "target"], axis=1)
    id_test = features_test["filename"]
    y_test = features_test["target"]
    features_test = features_test.drop(["filename", "target"], axis=1)
    
    #Feature transformation pipeline: only MinMaxScaling but more can be added later
    processing_pipeline = make_pipeline(MinMaxScaler())
    X_features = processing_pipeline.fit_transform(features_train)
    X_features_test = processing_pipeline.transform(features_test)
    
    #Training classifiers with default parameters
    for algo in classifiers:
        print("Training", classifier_name(algo))
        algo.fit(X_features, y_train)
        
    #Predicting with the trained classifiers and obtaining metrics
    for algo in classifiers:
        algo_name = classifier_name(algo)
        print("Predicting with", algo_name)
        y_pred = algo.predict(X_features_test)
        m = {"subset": str(feat_set), "model": algo_name, 
             "accuracy": accuracy_score(y_test, y_pred), 
             "F1": f1_score(y_test, y_pred), 
             "precision": precision_score(y_test, y_pred), 
             "recall": recall_score(y_test, y_pred)}
        metrics = metrics.append(m, ignore_index=True)

--- Processing Subset: ('FCTH',)
Set ('FCTH',) contains 194 features
Set ('FCTH',) contains 194 features
Training LogisticRegression
Training RandomForestClassifier
Training GaussianNB
Predicting with LogisticRegression
Predicting with RandomForestClassifier
Predicting with GaussianNB
--- Processing Subset: ('JCD',)
Set ('JCD',) contains 170 features
Set ('JCD',) contains 170 features
Training LogisticRegression
Training RandomForestClassifier
Training GaussianNB
Predicting with LogisticRegression
Predicting with RandomForestClassifier
Predicting with GaussianNB
--- Processing Subset: ('Tamura',)
Set ('Tamura',) contains 20 features
Set ('Tamura',) contains 20 features
Training LogisticRegression
Training RandomForestClassifier
Training GaussianNB
Predicting with LogisticRegression
Predicting with RandomForestClassifier
Predicting with GaussianNB
--- Processing Subset: ('FCTH', 'JCD')
Set ('FCTH', 'JCD') contains 362 features
Set ('FCTH', 'JCD') contains 362 features
Training LogisticR

In [8]:
metrics.set_index(["subset", "model"])

accuracy        F1  \
subset                    model                                        
('FCTH',)                 LogisticRegression      0.808638  0.572627   
                          RandomForestClassifier  0.816775  0.563449   
                          GaussianNB              0.621998  0.533071   
('JCD',)                  LogisticRegression      0.829123  0.630673   
                          RandomForestClassifier  0.813247  0.510733   
                          GaussianNB              0.726699  0.588890   
('Tamura',)               LogisticRegression      0.840560  0.688320   
                          RandomForestClassifier  0.802948  0.489459   
                          GaussianNB              0.829692  0.659693   
('FCTH', 'JCD')           LogisticRegression      0.822863  0.607885   
                          RandomForestClassifier  0.797713  0.458327   
                          GaussianNB              0.772676  0.621506   
('FCTH', 'Tamura')        LogisticRegression      0.824684  0.619583   
                          RandomForestClassifier  0.818425  0.542115   
                          GaussianNB              0.813588  0.697507   
('JCD', 'Tamura')         LogisticRegression      0.823660  0.609353   
                          RandomForestClassifier  0.811938  0.510298   
                          GaussianNB              0.797314  0.659334   
('FCTH', 'JCD', 'Tamura') LogisticRegression      0.821782  0.591016   
                          RandomForestClassifier  0.816149  0.513330   
                          GaussianNB              0.813588  0.667681   

                                                  precision    recall  
subset                    model                                        
('FCTH',)                 LogisticRegression       0.633577  0.522374  
                          RandomForestClassifier   0.678420  0.481799  
                          GaussianNB               0.382489  0.879202  
('JCD',)                  LogisticRegression       0.671556  0.594482  
                          RandomForestClassifier   0.715240  0.397171  
                          GaussianNB               0.466757  0.797589  
('Tamura',)               LogisticRegression       0.661535  0.717366  
                          RandomForestClassifier   0.672065  0.384883  
                          GaussianNB               0.647256  0.672618  
('FCTH', 'JCD')           LogisticRegression       0.665472  0.559471  
                          RandomForestClassifier   0.668444  0.348713  
                          GaussianNB               0.525473  0.760492  
('FCTH', 'Tamura')        LogisticRegression       0.662705  0.581730  
                          RandomForestClassifier   0.711220  0.437978  
                          GaussianNB               0.579561  0.875725  
('JCD', 'Tamura')         LogisticRegression       0.667680  0.560399  
                          RandomForestClassifier   0.706897  0.399258  
                          GaussianNB               0.561126  0.799212  
('FCTH', 'JCD', 'Tamura') LogisticRegression       0.676532  0.524693  
                          RandomForestClassifier   0.732588  0.395085  
                          GaussianNB               0.593508  0.763042

In [10]:
metrics.sort_values(by=["accuracy", "F1"], ascending=False).to_csv(dpath+"metrics.csv")